In [1]:
import dspy
from dspy.teleprompt import BootstrapFewShot 
import google.generativeai as genai
import weaviate
from dspy.retrieve.weaviate_rm import WeaviateRM
from dspy.teleprompt import BootstrapFewShot
import time
import json
import csv

c:\Users\PedroLustosa\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# Importando GEMINI_API_KEY
import os
from dotenv import load_dotenv
# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()
# Acessa a chave de API do Gemini
GEMINI_API_KEY  = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

In [3]:
with open('filtered_diseases_symptoms_Medlineplus.json', 'r') as arquivo:
    diseases_symptoms = json.load(arquivo)

In [4]:
# Lista de listas para armazenar o conteúdo do CSV
diseases = []
# Abrir e ler o arquivo CSV
with open('empty_symptoms_keys_Medlineplus.json', mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    
    # Ignorar o cabeçalho
    next(csv_reader)
    
    # Iterar sobre as linhas do CSV e adicionar a primeira coluna à lista
    for row in csv_reader:
        diseases.append(row[0]) 
        
diseases[0]

'    "Aspergillosis"'

In [5]:
# Criar uma lista de listas a partir do dicionário
diseases_symptoms_list = [[key, ', '.join(value)] for key, value in diseases_symptoms.items()]
diseases_symptoms_list[0]

['HIV',
 'Fever, Chills, Rash, Night sweats (heavy sweating during sleep), Muscle aches, Sore throat, Fatigue, Swollen lymph nodes, Mouth ulcers']

In [6]:
treino = [dspy.Example(question=diseases_symptoms_list[x][0], 
                       answer=diseases_symptoms_list[x][1]).with_inputs("question")
                                                 for x in range(len(diseases_symptoms_list))]

In [7]:
treino[0]

Example({'question': 'HIV', 'answer': 'Fever, Chills, Rash, Night sweats (heavy sweating during sleep), Muscle aches, Sore throat, Fatigue, Swollen lymph nodes, Mouth ulcers'}) (input_keys={'question'})

In [14]:
# Model Configuration
gemini_model = dspy.OpenAI(
    model="gemini-1.5-flash-latest",         # Specify the Gemini model version (e.g., "gemini-v1", "gemini-pro")
    model_type="chat",         # Set the model type to "chat" for conversational interactions
    max_tokens=400,            # Maximum number of tokens in the generated response
    temperature=0.4           # Controls the creativity/randomness of the model's output (0.0 - 1.0)
)
# DeepSpeed Configuration
dspy.settings.configure(lm=gemini_model)  

In [15]:
class CoT(dspy.Module):
    def __init__(self):
        self.sabatina = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.sabatina(question=question)

In [19]:
p_ = int(len(diseases_symptoms_list) * 1)
p_ =2

In [20]:
config = dict(max_bootstrapped_demos=p_, max_labeled_demos=p_)
otimizador = BootstrapFewShot(**config)

In [21]:
modelo = otimizador.compile(CoT(), trainset=treino)

 11%|█▏        | 4/35 [00:00<00:00, 618.29it/s]

Failed to run or to evaluate example Example({'question': 'HIV', 'answer': 'Fever, Chills, Rash, Night sweats (heavy sweating during sleep), Muscle aches, Sore throat, Fatigue, Swollen lymph nodes, Mouth ulcers'}) (input_keys={'question'}) with None due to The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable.
Failed to run or to evaluate example Example({'question': 'Acute Flaccid Myelitis', 'answer': 'Arm or leg weakness, A loss of muscle tone and reflexes, Facial drooping/weakness, Trouble moving the eyes, Drooping eyelids, Trouble swallowing, Slurred speech, Pain in the arms, legs, back, or neck'}) (input_keys={'question'}) with None due to The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable.
Failed to run or to evaluate example Example({'question': 'Bird Flu', 'answer': 'Fever (but not everyone has a fever), Cough, Sore throa

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable